In [15]:
!pip install Flask pyngrok


In [16]:
import sqlite3

def init_db():
    conn = sqlite3.connect('tasks.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS tasks (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            description TEXT NOT NULL,
            status INTEGER NOT NULL DEFAULT 0
        )
    ''')
    conn.commit()
    conn.close()

init_db()  # Initialize the database


In [10]:
from flask import Flask, request, redirect, render_template_string
from pyngrok import ngrok
import sqlite3

app = Flask(__name__)

def get_db_connection():
    conn = sqlite3.connect('tasks.db')
    conn.row_factory = sqlite3.Row  # To retrieve rows as dictionaries
    return conn


In [11]:
@app.route('/')
def index():
    conn = get_db_connection()
    tasks = conn.execute('SELECT * FROM tasks').fetchall()
    conn.close()
    return render_template_string('''
        <h1>Task List</h1>
        <a href="/create">Add New Task</a>
        <ul>
            {% for task in tasks %}
                <li>
                    {{ task['description'] }} -
                    {% if task['status'] %} Complete {% else %} Incomplete {% endif %}
                    <a href="/update/{{ task['id'] }}">Edit</a> |
                    <a href="/delete/{{ task['id'] }}">Delete</a>
                </li>
            {% endfor %}
        </ul>
    ''', tasks=tasks)


In [12]:
@app.route('/create', methods=('GET', 'POST'))
def create():
    if request.method == 'POST':
        description = request.form['description']
        conn = get_db_connection()
        conn.execute('INSERT INTO tasks (description) VALUES (?)', (description,))
        conn.commit()
        conn.close()
        return redirect('/')

    return render_template_string('''
        <h1>Create New Task</h1>
        <form method="post">
            <input type="text" name="description" placeholder="Task Description" required>
            <button type="submit">Create Task</button>
        </form>
        <a href="/">Back to Task List</a>
    ''')


In [13]:
@app.route('/update/<int:id>', methods=('GET', 'POST'))
def update(id):
    conn = get_db_connection()
    task = conn.execute('SELECT * FROM tasks WHERE id = ?', (id,)).fetchone()

    if request.method == 'POST':
        description = request.form['description']
        status = int(request.form['status'])
        conn.execute('UPDATE tasks SET description = ?, status = ? WHERE id = ?', (description, status, id))
        conn.commit()
        conn.close()
        return redirect('/')

    conn.close()
    return render_template_string('''
        <h1>Update Task</h1>
        <form method="post">
            <input type="text" name="description" value="{{ task['description'] }}" required>
            <label>Status:</label>
            <select name="status">
                <option value="0" {% if task['status'] == 0 %} selected {% endif %}>Incomplete</option>
                <option value="1" {% if task['status'] == 1 %} selected {% endif %}>Complete</option>
            </select>
            <button type="submit">Update Task</button>
        </form>
        <a href="/">Back to Task List</a>
    ''', task=task)


In [14]:
@app.route('/delete/<int:id>', methods=('GET', 'POST'))
def delete(id):
    conn = get_db_connection()
    if request.method == 'POST':
        conn.execute('DELETE FROM tasks WHERE id = ?', (id,))
        conn.commit()
        conn.close()
        return redirect('/')

    task = conn.execute('SELECT * FROM tasks WHERE id = ?', (id,)).fetchone()
    conn.close()
    return render_template_string('''
        <h1>Delete Task</h1>
        <p>Are you sure you want to delete this task?</p>
        <p>{{ task['description'] }}</p>
        <form method="post">
            <button type="submit">Delete</button>
        </form>
        <a href="/">Cancel</a>
    ''', task=task)


In [18]:
# Start the app and expose it with ngrok
port = 5000
ngrok_tunnel = ngrok.connect(port)
print(" * ngrok URL:", ngrok_tunnel.public_url)

app.run(port=port)


 * ngrok URL: https://6877-34-16-211-44.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Nov/2024 13:31:18] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2024 13:31:19] "GET /favicon.ico HTTP/1.1" 404 -


In [17]:
!ngrok config add-authtoken 2oHnLafkdMc0OgdRmzNLwzy42Mi_2Pwy2YrAzV8dTiKTHKXMm


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
